In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.metrics.pairwise import euclidean_distances
%matplotlib widget


In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X_names = df.columns[df.columns != 'Outcome']
X = df[X_names]
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X) #estandarizamos nuestra matriz de diseño
X_scaled = pd.DataFrame(X_scaled, columns = X.columns)

y = df['Outcome']
X_scaled

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [4]:
X_scaled.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-6.476301e-17,-9.251859e-18,1.503427e-17,1.006140e-16,-3.006854e-17,2.590520e-16,2.451743e-16,1.931325e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


In [5]:
def Squared_exponential_kernel(X, Y, l):
    kernel = euclidean_distances(X, Y) ** 2
    kernel = kernel*(-1/(l**2))
    kernel = np.exp(kernel)
    return kernel

def Orstein_Uhlenbeck_kernel(X, Y, l):
    kernel = euclidean_distances(X, Y)
    kernel = kernel*(-1/(l**2))
    kernel = np.exp(kernel)
    return kernel

def Rational_quadratic_kernel(X, Y, l, sigma, alpha):
    kernel = euclidean_distances(X, Y) ** 2
    kernel = 1 + kernel/(2*alpha*l**2)
    kernel = sigma*kernel**(-alpha)
    return kernel

def Periodic_kernel(X, Y, sigma, p, l):
    kernel = np.pi*euclidean_distances(X, Y)*(1/p)
    kernel = -2*(np.sin(kernel)**2)*(1/l**2)
    kernel = (sigma**2)*np.exp(kernel)
    return kernel

def Polynomial_kernel(X, Y, sigmab, sigmav, mu, alpha):
    kernel = sigmab**2 + (sigmav**2)*(X-mu)@(Y-mu).T
    kernel = kernel**alpha
    return kernel

In [8]:
ls, cs = np.logspace(0.0, 2.0, num=50, base=8), np.arange(10,810,50)
precisionCV, recallCV, fscoreCV, accuracyCV = [],[],[],[]
precisionCV_SD, recallCV_SD, fscoreCV_SD, accuracyCV_SD = [],[],[],[]
ls_comb, cs_comb = [], []

for l_sigma in ls:
    for c in cs:
        precisions, recalls, fscores, accuracys = [], [], [], []
        #Haremos stratified K fold con K=4
        skf = StratifiedKFold(n_splits=4)
        for train_index, test_index in skf.split(X_scaled, y):
            #Separamos train y test set
            X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            #hacemos nuestras matriz de Gram de entrenamiento y predictiva, pasando el sigma del loop como parametro
            matriz_gram_train=Squared_exponential_kernel(X_train,X_train,l=l_sigma)
            matriz_gram_test =Squared_exponential_kernel(X_test,X_train,l=l_sigma)  

            # we create an instance of SVM and fit out data.
            clf = svm.SVC(C=c, kernel='precomputed')

            clf.fit(matriz_gram_train, y_train)
            y_pred = clf.predict(matriz_gram_test)

            report = precision_recall_fscore_support(y_test, y_pred, average='binary')
            precision, recall, fscore = report[0], report[1], report[2]
            accuracy = accuracy_score(y_test, y_pred)
            
            precisions.append(precision)
            recalls.append(recall)
            fscores.append(fscore)
            accuracys.append(accuracy)
            
        precisionCV.append(np.mean(precisions))
        recallCV.append(np.mean(recalls))
        fscoreCV.append(np.mean(fscores))
        accuracyCV.append(np.mean(accuracys))

        precisionCV_SD.append(np.std(precisions))
        recallCV_SD.append(np.std(recalls))
        fscoreCV_SD.append(np.std(fscores))
        accuracyCV_SD.append(np.std(accuracys))
        
        ls_comb.append(l_sigma)
        cs_comb.append(c)
        #print(l_sigma,c)

In [7]:
df = pd.DataFrame(precisionCV, columns=['AVG Precision'])
df['AVG recall'] = recallCV
df['AVG fscore'] = fscoreCV
df['AVG accuracy'] = accuracyCV
df['SD Precision'] = precisionCV_SD
df['SD recall'] = recallCV_SD
df['SD fscore'] = fscoreCV_SD
df['SD accuracy'] = accuracyCV_SD
df['l'] = ls_comb
df['C'] = cs_comb
df

,AVG Precision,AVG recall,AVG fscore,AVG accuracy,SD Precision,SD recall,SD fscore,SD accuracy,l,C
0,0.622036,0.466418,0.531505,0.713542,0.059949,0.048794,0.044781,0.028048,1.0,10
1,0.618775,0.466418,0.530438,0.712240,0.061397,0.048794,0.045979,0.028852,1.0,60
2,0.618775,0.466418,0.530438,0.712240,0.061397,0.048794,0.045979,0.028852,1.0,110
3,0.618775,0.466418,0.530438,0.712240,0.061397,0.048794,0.045979,0.028852,1.0,160
4,0.618775,0.466418,0.530438,0.712240,0.061397,0.048794,0.045979,0.028852,1.0,210
...,...,...,...,...,...,...,...,...,...,...
795,0.711600,0.555970,0.623803,0.765625,0.048246,0.022075,0.029218,0.021788,64.0,560
796,0.715047,0.555970,0.625133,0.766927,0.048995,0.022075,0.029739,0.022212,64.0,610
797,0.715047,0.555970,0.625133,0.766927,0.048995,0.022075,0.029739,0.022212,64.0,660
798,0.715047,0.555970,0.625133,0.766927,0.048995,0.022075,0.029739,0.022212,64.0,710


In [8]:
max_Precision = df.iloc[df['AVG Precision'].idxmax(),:]
max_recall = df.iloc[df['AVG recall'].idxmax(),:]
max_fscore = df.iloc[df['AVG fscore'].idxmax(),:]
max_accuracy = df.iloc[df['AVG accuracy'].idxmax(),:]

In [9]:
b_model = pd.DataFrame()
b_model['max_Precision'] = max_Precision
b_model['max_recall'] = max_recall
b_model['max_fscore'] = max_fscore
b_model['max_accuracy'] = max_accuracy
b_model

,max_Precision,max_recall,max_fscore,max_accuracy
AVG Precision,0.810233,0.654275,0.716460,0.745783
AVG recall,0.432836,0.608209,0.593284,0.563433
AVG fscore,0.563116,0.629411,0.648715,0.641501
AVG accuracy,0.765625,0.750000,0.776042,0.779948
SD Precision,0.060501,0.078353,0.074958,0.046716
SD recall,0.023599,0.070304,0.067886,0.022075
SD fscore,0.024820,0.070651,0.069867,0.028410
SD accuracy,0.015625,0.048440,0.044500,0.020629
l,64.000000,3.572006,3.281341,13.889458
C,10.000000,110.000000,10.000000,10.000000


In [16]:
sigmas, ps = np.arange(1,10,3), np.arange(1,10,3)
cs, ls = np.arange(10,810,50), np.logspace(0.0, 2.0, num=30, base=4)

precisionCV, recallCV, fscoreCV, accuracyCV = [],[],[],[]
precisionCV_SD, recallCV_SD, fscoreCV_SD, accuracyCV_SD = [],[],[],[]

sigmas_comb, ps_comb, ls_comb, cs_comb = [], [], [], []

for sigma in sigmas:
    for p in ps:
        for l in ls:
            for c in cs:
                precisions, recalls, fscores, accuracys = [], [], [], []
                #Haremos stratified K fold con K=4
                skf = StratifiedKFold(n_splits=4)
                for train_index, test_index in skf.split(X_scaled, y):
                    #Separamos train y test set
                    X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                    #hacemos nuestras matriz de Gram de entrenamiento y predictiva, pasando el sigma del loop como parametro
                    matriz_gram_train=Periodic_kernel(X_train,X_train, sigma, p, l)
                    matriz_gram_test =Periodic_kernel(X_test, X_train, sigma, p, l)  

                    # we create an instance of SVM and fit out data.
                    clf = svm.SVC(C=c, kernel='precomputed')
                    
                    clf.fit(matriz_gram_train, y_train)
                    y_pred = clf.predict(matriz_gram_test)

                    report = precision_recall_fscore_support(y_test, y_pred, average='binary')
                    precision, recall, fscore = report[0], report[1], report[2]
                    accuracy = accuracy_score(y_test, y_pred)

                    precisions.append(precision)
                    recalls.append(recall)
                    fscores.append(fscore)
                    accuracys.append(accuracy)

                precisionCV.append(np.mean(precisions))
                recallCV.append(np.mean(recalls))
                fscoreCV.append(np.mean(fscores))
                accuracyCV.append(np.mean(accuracys))

                precisionCV_SD.append(np.std(precisions))
                recallCV_SD.append(np.std(recalls))
                fscoreCV_SD.append(np.std(fscores))
                accuracyCV_SD.append(np.std(accuracys))

                cs_comb.append(c)
                sigmas_comb.append(sigma)
                ps_comb.append(p)
                ls_comb.append(l)
                print(sigma, p, l,c)

1 1 1.0 10
1 1 1.0 60
1 1 1.0 110
1 1 1.0 160
1 1 1.0 210
1 1 1.0 260
1 1 1.0 310
1 1 1.0 360
1 1 1.0 410
1 1 1.0 460
1 1 1.0 510
1 1 1.0 560
1 1 1.0 610
1 1 1.0 660
1 1 1.0 710
1 1 1.0 760
1 1 1.1003260089454083 10
1 1 1.1003260089454083 60
1 1 1.1003260089454083 110
1 1 1.1003260089454083 160
1 1 1.1003260089454083 210
1 1 1.1003260089454083 260
1 1 1.1003260089454083 310
1 1 1.1003260089454083 360
1 1 1.1003260089454083 410
1 1 1.1003260089454083 460
1 1 1.1003260089454083 510
1 1 1.1003260089454083 560
1 1 1.1003260089454083 610
1 1 1.1003260089454083 660
1 1 1.1003260089454083 710
1 1 1.1003260089454083 760
1 1 1.2107173259617308 10
1 1 1.2107173259617308 60
1 1 1.2107173259617308 110
1 1 1.2107173259617308 160
1 1 1.2107173259617308 210
1 1 1.2107173259617308 260
1 1 1.2107173259617308 310
1 1 1.2107173259617308 360
1 1 1.2107173259617308 410
1 1 1.2107173259617308 460
1 1 1.2107173259617308 510
1 1 1.2107173259617308 560
1 1 1.2107173259617308 610
1 1 1.2107173259617308 660
1 1 

1 1 6.767489244843735 60
1 1 6.767489244843735 110
1 1 6.767489244843735 160
1 1 6.767489244843735 210
1 1 6.767489244843735 260
1 1 6.767489244843735 310
1 1 6.767489244843735 360
1 1 6.767489244843735 410
1 1 6.767489244843735 460
1 1 6.767489244843735 510
1 1 6.767489244843735 560
1 1 6.767489244843735 610
1 1 6.767489244843735 660
1 1 6.767489244843735 710
1 1 6.767489244843735 760
1 1 7.446444431359882 10
1 1 7.446444431359882 60
1 1 7.446444431359882 110
1 1 7.446444431359882 160
1 1 7.446444431359882 210
1 1 7.446444431359882 260
1 1 7.446444431359882 310
1 1 7.446444431359882 360
1 1 7.446444431359882 410
1 1 7.446444431359882 460
1 1 7.446444431359882 510
1 1 7.446444431359882 560
1 1 7.446444431359882 610
1 1 7.446444431359882 660
1 1 7.446444431359882 710
1 1 7.446444431359882 760
1 1 8.193516481991978 10
1 1 8.193516481991978 60
1 1 8.193516481991978 110
1 1 8.193516481991978 160
1 1 8.193516481991978 210
1 1 8.193516481991978 260
1 1 8.193516481991978 310
1 1 8.19351648199

1 4 2.601439840712011 510
1 4 2.601439840712011 560
1 4 2.601439840712011 610
1 4 2.601439840712011 660
1 4 2.601439840712011 710
1 4 2.601439840712011 760
1 4 2.8624319174422257 10
1 4 2.8624319174422257 60
1 4 2.8624319174422257 110
1 4 2.8624319174422257 160
1 4 2.8624319174422257 210
1 4 2.8624319174422257 260
1 4 2.8624319174422257 310
1 4 2.8624319174422257 360
1 4 2.8624319174422257 410
1 4 2.8624319174422257 460
1 4 2.8624319174422257 510
1 4 2.8624319174422257 560
1 4 2.8624319174422257 610
1 4 2.8624319174422257 660
1 4 2.8624319174422257 710
1 4 2.8624319174422257 760
1 4 3.149608287597157 10
1 4 3.149608287597157 60
1 4 3.149608287597157 110
1 4 3.149608287597157 160
1 4 3.149608287597157 210
1 4 3.149608287597157 260
1 4 3.149608287597157 310
1 4 3.149608287597157 360
1 4 3.149608287597157 410
1 4 3.149608287597157 460
1 4 3.149608287597157 510
1 4 3.149608287597157 560
1 4 3.149608287597157 610
1 4 3.149608287597157 660
1 4 3.149608287597157 710
1 4 3.149608287597157 760


1 7 1.1003260089454083 260
1 7 1.1003260089454083 310
1 7 1.1003260089454083 360
1 7 1.1003260089454083 410
1 7 1.1003260089454083 460
1 7 1.1003260089454083 510
1 7 1.1003260089454083 560
1 7 1.1003260089454083 610
1 7 1.1003260089454083 660
1 7 1.1003260089454083 710
1 7 1.1003260089454083 760
1 7 1.2107173259617308 10
1 7 1.2107173259617308 60
1 7 1.2107173259617308 110
1 7 1.2107173259617308 160
1 7 1.2107173259617308 210
1 7 1.2107173259617308 260
1 7 1.2107173259617308 310
1 7 1.2107173259617308 360
1 7 1.2107173259617308 410
1 7 1.2107173259617308 460
1 7 1.2107173259617308 510
1 7 1.2107173259617308 560
1 7 1.2107173259617308 610
1 7 1.2107173259617308 660
1 7 1.2107173259617308 710
1 7 1.2107173259617308 760
1 7 1.332183763236528 10
1 7 1.332183763236528 60
1 7 1.332183763236528 110
1 7 1.332183763236528 160
1 7 1.332183763236528 210
1 7 1.332183763236528 260
1 7 1.332183763236528 310
1 7 1.332183763236528 360
1 7 1.332183763236528 410
1 7 1.332183763236528 460
1 7 1.332183763

1 7 6.767489244843735 760
1 7 7.446444431359882 10
1 7 7.446444431359882 60
1 7 7.446444431359882 110
1 7 7.446444431359882 160
1 7 7.446444431359882 210
1 7 7.446444431359882 260
1 7 7.446444431359882 310
1 7 7.446444431359882 360
1 7 7.446444431359882 410
1 7 7.446444431359882 460
1 7 7.446444431359882 510
1 7 7.446444431359882 560
1 7 7.446444431359882 610
1 7 7.446444431359882 660
1 7 7.446444431359882 710
1 7 7.446444431359882 760
1 7 8.193516481991978 10
1 7 8.193516481991978 60
1 7 8.193516481991978 110
1 7 8.193516481991978 160
1 7 8.193516481991978 210
1 7 8.193516481991978 260
1 7 8.193516481991978 310
1 7 8.193516481991978 360
1 7 8.193516481991978 410
1 7 8.193516481991978 460
1 7 8.193516481991978 510
1 7 8.193516481991978 560
1 7 8.193516481991978 610
1 7 8.193516481991978 660
1 7 8.193516481991978 710
1 7 8.193516481991978 760
1 7 9.015539289858657 10
1 7 9.015539289858657 60
1 7 9.015539289858657 110
1 7 9.015539289858657 160
1 7 9.015539289858657 210
1 7 9.015539289858

...

7 1 1.2107173259617308 760
7 1 1.332183763236528 10
7 1 1.332183763236528 60
7 1 1.332183763236528 110
7 1 1.332183763236528 160
7 1 1.332183763236528 210
7 1 1.332183763236528 260
7 1 1.332183763236528 310
7 1 1.332183763236528 360
7 1 1.332183763236528 410
7 1 1.332183763236528 460
7 1 1.332183763236528 510
7 1 1.332183763236528 560
7 1 1.332183763236528 610
7 1 1.332183763236528 660
7 1 1.332183763236528 710
7 1 1.332183763236528 760
7 1 1.4658364433839237 10
7 1 1.4658364433839237 60
7 1 1.4658364433839237 110
7 1 1.4658364433839237 160
7 1 1.4658364433839237 210
7 1 1.4658364433839237 260
7 1 1.4658364433839237 310
7 1 1.4658364433839237 360
7 1 1.4658364433839237 410
7 1 1.4658364433839237 460
7 1 1.4658364433839237 510
7 1 1.4658364433839237 560
7 1 1.4658364433839237 610
7 1 1.4658364433839237 660
7 1 1.4658364433839237 710
7 1 1.4658364433839237 760
7 1 1.6128979635153649 10
7 1 1.6128979635153649 60
7 1 1.6128979635153649 110
7 1 1.6128979635153649 160
7 1 1.6128979635153649 

7 1 8.193516481991978 460
7 1 8.193516481991978 510
7 1 8.193516481991978 560
7 1 8.193516481991978 610
7 1 8.193516481991978 660
7 1 8.193516481991978 710
7 1 8.193516481991978 760
7 1 9.015539289858657 10
7 1 9.015539289858657 60
7 1 9.015539289858657 110
7 1 9.015539289858657 160
7 1 9.015539289858657 210
7 1 9.015539289858657 260
7 1 9.015539289858657 310
7 1 9.015539289858657 360
7 1 9.015539289858657 410
7 1 9.015539289858657 460
7 1 9.015539289858657 510
7 1 9.015539289858657 560
7 1 9.015539289858657 610
7 1 9.015539289858657 660
7 1 9.015539289858657 710
7 1 9.015539289858657 760
7 1 9.920032365300695 10
7 1 9.920032365300695 60
7 1 9.920032365300695 110
7 1 9.920032365300695 160
7 1 9.920032365300695 210
7 1 9.920032365300695 260
7 1 9.920032365300695 310
7 1 9.920032365300695 360
7 1 9.920032365300695 410
7 1 9.920032365300695 460
7 1 9.920032365300695 510
7 1 9.920032365300695 560
7 1 9.920032365300695 610
7 1 9.920032365300695 660
7 1 9.920032365300695 710
7 1 9.9200323653

7 4 3.465595916833162 60
7 4 3.465595916833162 110
7 4 3.465595916833162 160
7 4 3.465595916833162 210
7 4 3.465595916833162 260
7 4 3.465595916833162 310
7 4 3.465595916833162 360
7 4 3.465595916833162 410
7 4 3.465595916833162 460
7 4 3.465595916833162 510
7 4 3.465595916833162 560
7 4 3.465595916833162 610
7 4 3.465595916833162 660
7 4 3.465595916833162 710
7 4 3.465595916833162 760
7 4 3.8132853237865354 10
7 4 3.8132853237865354 60
7 4 3.8132853237865354 110
7 4 3.8132853237865354 160
7 4 3.8132853237865354 210
7 4 3.8132853237865354 260
7 4 3.8132853237865354 310
7 4 3.8132853237865354 360
7 4 3.8132853237865354 410
7 4 3.8132853237865354 460
7 4 3.8132853237865354 510
7 4 3.8132853237865354 560
7 4 3.8132853237865354 610
7 4 3.8132853237865354 660
7 4 3.8132853237865354 710
7 4 3.8132853237865354 760
7 4 4.195857021292138 10
7 4 4.195857021292138 60
7 4 4.195857021292138 110
7 4 4.195857021292138 160
7 4 4.195857021292138 210
7 4 4.195857021292138 260
7 4 4.195857021292138 310
7

7 7 1.332183763236528 610
7 7 1.332183763236528 660
7 7 1.332183763236528 710
7 7 1.332183763236528 760
7 7 1.4658364433839237 10
7 7 1.4658364433839237 60
7 7 1.4658364433839237 110
7 7 1.4658364433839237 160
7 7 1.4658364433839237 210
7 7 1.4658364433839237 260
7 7 1.4658364433839237 310
7 7 1.4658364433839237 360
7 7 1.4658364433839237 410
7 7 1.4658364433839237 460
7 7 1.4658364433839237 510
7 7 1.4658364433839237 560
7 7 1.4658364433839237 610
7 7 1.4658364433839237 660
7 7 1.4658364433839237 710
7 7 1.4658364433839237 760
7 7 1.6128979635153649 10
7 7 1.6128979635153649 60
7 7 1.6128979635153649 110
7 7 1.6128979635153649 160
7 7 1.6128979635153649 210
7 7 1.6128979635153649 260
7 7 1.6128979635153649 310
7 7 1.6128979635153649 360
7 7 1.6128979635153649 410
7 7 1.6128979635153649 460
7 7 1.6128979635153649 510
7 7 1.6128979635153649 560
7 7 1.6128979635153649 610
7 7 1.6128979635153649 660
7 7 1.6128979635153649 710
7 7 1.6128979635153649 760
7 7 1.7747135790310382 10
7 7 1.7747

7 7 9.015539289858657 310
7 7 9.015539289858657 360
7 7 9.015539289858657 410
7 7 9.015539289858657 460
7 7 9.015539289858657 510
7 7 9.015539289858657 560
7 7 9.015539289858657 610
7 7 9.015539289858657 660
7 7 9.015539289858657 710
7 7 9.015539289858657 760
7 7 9.920032365300695 10
7 7 9.920032365300695 60
7 7 9.920032365300695 110
7 7 9.920032365300695 160
7 7 9.920032365300695 210
7 7 9.920032365300695 260
7 7 9.920032365300695 310
7 7 9.920032365300695 360
7 7 9.920032365300695 410
7 7 9.920032365300695 460
7 7 9.920032365300695 510
7 7 9.920032365300695 560
7 7 9.920032365300695 610
7 7 9.920032365300695 660
7 7 9.920032365300695 710
7 7 9.920032365300695 760
7 7 10.915269621120592 10
7 7 10.915269621120592 60
7 7 10.915269621120592 110
7 7 10.915269621120592 160
7 7 10.915269621120592 210
7 7 10.915269621120592 260
7 7 10.915269621120592 310
7 7 10.915269621120592 360
7 7 10.915269621120592 410
7 7 10.915269621120592 460
7 7 10.915269621120592 510
7 7 10.915269621120592 560
7 7 

In [17]:
df = pd.DataFrame(precisionCV, columns=['AVG Precision'])
df['AVG recall'] = recallCV
df['AVG fscore'] = fscoreCV
df['AVG accuracy'] = accuracyCV
df['SD Precision'] = precisionCV_SD
df['SD recall'] = recallCV_SD
df['SD fscore'] = fscoreCV_SD
df['SD accuracy'] = accuracyCV_SD
df['sigma'] = sigmas_comb
df['p'] = ps_comb
df['l'] = ls_comb
df['C'] = cs_comb
df

,AVG Precision,AVG recall,AVG fscore,AVG accuracy,SD Precision,SD recall,SD fscore,SD accuracy,sigma,p,l,C
0,0.392260,0.347015,0.367110,0.584635,0.032174,0.048794,0.036652,0.019618,1,1,1.0,10
1,0.361978,0.320896,0.338645,0.567708,0.029553,0.061990,0.046312,0.014264,1,1,1.0,60
2,0.349706,0.298507,0.319765,0.563802,0.034470,0.069206,0.051564,0.018914,1,1,1.0,110
3,0.349706,0.298507,0.319765,0.563802,0.034470,0.069206,0.051564,0.018914,1,1,1.0,160
4,0.337811,0.291045,0.310356,0.555990,0.018240,0.065485,0.044282,0.008538,1,1,1.0,210
...,...,...,...,...,...,...,...,...,...,...,...,...
4315,0.253215,0.250000,0.250907,0.483073,0.019701,0.042707,0.030362,0.011276,7,7,16.0,560
4316,0.253215,0.250000,0.250907,0.483073,0.019701,0.042707,0.030362,0.011276,7,7,16.0,610
4317,0.253215,0.250000,0.250907,0.483073,0.019701,0.042707,0.030362,0.011276,7,7,16.0,660
4318,0.253215,0.250000,0.250907,0.483073,0.019701,0.042707,0.030362,0.011276,7,7,16.0,710


In [18]:
max_Precision = df.iloc[df['AVG Precision'].idxmax(),:]
max_recall = df.iloc[df['AVG recall'].idxmax(),:]
max_fscore = df.iloc[df['AVG fscore'].idxmax(),:]
max_accuracy = df.iloc[df['AVG accuracy'].idxmax(),:]

In [19]:
b_model = pd.DataFrame()
b_model['max_Precision'] = max_Precision
b_model['max_recall'] = max_recall
b_model['max_fscore'] = max_fscore
b_model['max_accuracy'] = max_accuracy
b_model

,max_Precision,max_recall,max_fscore,max_accuracy
AVG Precision,0.392260,0.371995,0.392260,0.204167
AVG recall,0.347015,0.361940,0.347015,0.018657
AVG fscore,0.367110,0.366064,0.367110,0.034072
AVG accuracy,0.584635,0.561198,0.584635,0.632812
SD Precision,0.032174,0.035567,0.032174,0.144518
SD recall,0.048794,0.006463,0.048794,0.012375
SD fscore,0.036652,0.017141,0.036652,0.022572
SD accuracy,0.019618,0.030675,0.019618,0.007813
sigma,1.000000,1.000000,1.000000,1.000000
p,1.000000,1.000000,1.000000,1.000000


In [26]:
X_k = np.arange(-4,4,0.1).reshape(-1, 1)

sigma, l, alpha = 1, 1, 2
p = 3
sigmab, sigmav, mu, alpha = 2,2,1,2

K1 = Squared_exponential_kernel(X_k, X_k, l)
K2 = Orstein_Uhlenbeck_kernel(X_k, X_k, l)
K3 = Rational_quadratic_kernel(X_k, X_k, sigma, l, alpha)
K4 = Periodic_kernel(X_k, X_k, sigma, p, l)
K5 = Polynomial_kernel(X_k, X_k, sigmab, sigmav, mu, alpha)

xs,ys = [],[]
zs1, zs2, zs3, zs4, zs5 = [],[],[],[],[]

for i in range(K2.shape[0]):
    for j in range(K2.shape[1]):
        xs.append(X_k[i])
        ys.append(X_k[j])
        zs1.append(K1[i,j])
        zs2.append(K2[i,j])
        zs3.append(K3[i,j])
        zs4.append(K4[i,j])
        zs5.append(K5[i,j])

In [27]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs,ys,zs1, c='green')
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs,ys,zs2, c='green')
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs,ys,zs3, c='green')
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs,ys,zs4, c='green')
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xs,ys,zs5, c='green')
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#print(report)
#Create confusion matrix
#print("Confusion matrix")
#ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='Reds')
#lt.show()